------------

<h2>Notebook to transform each flight into one line and agregating all of them to create one parquet file for each aircraft</h2>

In [4]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import gc
import os
import copy

In [2]:
# Separating the columns that are described in the TAP
columns_needed = ['amscHprsovDrivF-1a', 'amscHprsovDrivF-1b',
                'amscHprsovDrivF-2b', 'amscPrsovDrivF-1a',
                'amscPrsovDrivF-1b', 'amscPrsovDrivF-2b',
                'basBleedLowPressF-1a', 'basBleedLowPressF-2b',
                'basBleedLowTempF-1a', 'basBleedLowTempF-2b',
                'basBleedOverPressF-1a', 'basBleedOverPressF-2b',
                'basBleedOverTempF-1a', 'basBleedOverTempF-2b',
                'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b',
                'bleedFavTmCmd-2a', 'bleedFavTmCmd-2b', 'bleedFavTmFbk-1a',
                'bleedFavTmFbk-1b', 'bleedFavTmFbk-2b', 'bleedHprsovCmdStatus-1a',
                'bleedHprsovCmdStatus-1b', 'bleedHprsovCmdStatus-2a',
                'bleedHprsovCmdStatus-2b', 'bleedHprsovOpPosStatus-1a',
                'bleedHprsovOpPosStatus-1b', 'bleedHprsovOpPosStatus-2a',
                'bleedHprsovOpPosStatus-2b', 'bleedMonPress-1a',
                'bleedMonPress-1b', 'bleedMonPress-2a', 'bleedMonPress-2b',
                'bleedOnStatus-1a', 'bleedOnStatus-1b', 'bleedOnStatus-2b',
                'bleedOverpressCas-2a', 'bleedOverpressCas-2b',
                'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                'bleedPrecoolDiffPress-2a', 'bleedPrecoolDiffPress-2b',
                'bleedPrsovClPosStatus-1a', 'bleedPrsovClPosStatus-2a',
                'bleedPrsovFbk-1a', 'bleedPrsovFbk-1b', 'bleedPrsovFbk-2b', 'message0418DAA-1', 'message0422DAA-1']

In [3]:
# Creating a dictionary to map the column names to the aggregation method names
methods = {
    'amscHprsovDrivF-1a': 'sum',
    'amscHprsovDrivF-1b': 'sum',
    'amscHprsovDrivF-2b': 'sum',
    'amscPrsovDrivF-1a': 'sum',
    'amscPrsovDrivF-1b': 'sum', 
    'amscPrsovDrivF-2b': 'sum',
    'basBleedLowPressF-1a': 'sum',
    'basBleedLowPressF-2b': 'sum',
    'basBleedLowTempF-1a': 'sum',
    'basBleedLowTempF-2b': 'sum',
    'basBleedOverPressF-1a': 'sum', 
    'basBleedOverPressF-2b': 'sum',
    'basBleedOverTempF-1a': 'sum', 
    'basBleedOverTempF-2b': 'sum',
    'bleedFavTmCmd-1a': 'mode', 
    'bleedFavTmCmd-1b': 'mode',
    'bleedFavTmCmd-2a': 'mode', 
    'bleedFavTmCmd-2b': 'mode', 
    'bleedFavTmFbk-1a': 'mode',
    'bleedFavTmFbk-1b': 'mode', 
    'bleedFavTmFbk-2b': 'mode', 
    'bleedHprsovCmdStatus-1a': 'sum',
    'bleedHprsovCmdStatus-1b': 'sum', 
    'bleedHprsovCmdStatus-2a': 'sum',
    'bleedHprsovCmdStatus-2b': 'sum', 
    'bleedHprsovOpPosStatus-1a': 'sum',
    'bleedHprsovOpPosStatus-1b': 'sum', 
    'bleedHprsovOpPosStatus-2a': 'sum',
    'bleedHprsovOpPosStatus-2b': 'sum', 
    'bleedMonPress-1a': 'mode',
    'bleedMonPress-1b': 'mode', 
    'bleedMonPress-2a': 'mode', 
    'bleedMonPress-2b': 'mode',
    'bleedOnStatus-1a': 'sum', 
    'bleedOnStatus-1b': 'sum', 
    'bleedOnStatus-2b': 'sum',
    'bleedOverpressCas-2a': 'sum', 
    'bleedOverpressCas-2b': 'sum',
    'bleedPrecoolDiffPress-1a': 'mean', 
    'bleedPrecoolDiffPress-1b': 'mean',
    'bleedPrecoolDiffPress-2a': 'mean', 
    'bleedPrecoolDiffPress-2b': 'mean',
    'bleedPrsovClPosStatus-1a': 'sum',
    'bleedPrsovClPosStatus-2a': 'sum',
    'bleedPrsovFbk-1a': 'sum',
    'bleedPrsovFbk-1b': 'sum',
    'bleedPrsovFbk-2b': 'sum',
    'message0418DAA-1': 'answer',
    'message0422DAA-1': 'answer',
}

In [4]:
# Reduces the size of the dataframe by converting columns to smaller data types
def convert_columns(df_filtered):
    for column in df_filtered.columns:
        if df_filtered[column].dtype == 'float64':
            df_filtered[column] = df_filtered[column].astype('float32')

        if df_filtered[column].dtype == 'int64':
            df_filtered[column] = df_filtered[column].astype('int32')

    return df_filtered

In [5]:
# Agregation of the colunms based on the method
def process_column(col_name, method, dataframe):
    if method == 'max':
        return dataframe[col_name].max()
    elif method == 'sum':
        return dataframe[col_name].sum()
    elif method == 'answer':
        dataframe[col_name] = dataframe[col_name].apply(lambda x: 1 if x != 0 and pd.notnull(x) else 0)
        return dataframe[col_name].sum()
    elif method == 'mode':
        return dataframe[col_name].mode()[0]
    elif method == 'mean':
        return dataframe[col_name].mean()
    else:
        raise ValueError(f"Unknown method {method} for column {col_name}")

In [10]:
# Function to aggregate the data of one folder
def process_files_in_directory(dirpath, methods):
    # Formating the output file name
    output_file = os.path.join(output_directory, os.path.basename(dirpath) + "_consolidated.parquet")

    # If the output file already exists, read it, otherwise create an empty DataFrame
    if os.path.exists(output_file):
        df_output = pd.read_parquet(output_file)
    else:
        df_output = pd.DataFrame()

    # Loop through all files in the directory and process them
    for filename in os.listdir(dirpath):
        if filename.endswith('.parquet'):
            file_path = os.path.join(dirpath, filename)
            count = 0
            
            try:
                # Load the file
                df = pd.read_parquet(file_path)

                count += len(df) 
                # Select only the columns we need
                df = df.loc[:, columns_needed]
                # Reducing the data types to save memory
                df = convert_columns(df)

                count = len(df)
                
                # Summarizing the data and creating a new DataFrame with the results
                summary_data = copy.deepcopy({col: process_column(col, method, df) for col, method in methods.items()})

                
                df_resumo = pd.DataFrame([summary_data])

                # Creating a new colum called fightHours that receives COUNT
                df_resumo['flightMinutes'] = count * 50 / 60000
                
                # Agregating the previous results with the new ones
                df_output = pd.concat([df_output, df_resumo], ignore_index=True)
                
                # Cleaning up the memory
                del df
                gc.collect()

            except Exception as e:
                print(f"Erro ao ler o arquivo {file_path}: {e}")

    # Salve o DataFrame de saída no arquivo Parquet de saída
    df_output.to_parquet(output_file)

    # Cleaning up the memory
    del df_output
    gc.collect()

In [11]:
# Paths to the data files
directory_path = "/Users/henriquematias/Documents/GitHub/Inteli-Modulo-7/Projeto_Grupo1_Inteli_Azul/notebooks/Datasets"
output_directory = "/Users/henriquematias/Documents/GitHub/Inteli-Modulo-7/Projeto_Grupo1_Inteli_Azul/notebooks/Output/Voos_em_1_linha"

# Making sure that the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Passing through all the files in all directories specified in the directory_path
for dirpath, dirnames, filenames in os.walk(directory_path):
    process_files_in_directory(dirpath, methods)

Erro ao ler o arquivo /Users/henriquematias/Documents/GitHub/Inteli-Modulo-7/Projeto_Grupo1_Inteli_Azul/notebooks/Datasets/06120025/TCRF_ARCHIVE_06120025_20230501075641.parquet: Couldn't deserialize thrift: don't know what type: 
Deserializing page header failed.



------

## Normalização

In [5]:
PATH_DF_CONSOLIDADO = 'C:/Users/Inteli/Desktop/testes-etl/output/06120089_consolidated.parquet'

df_consolidado = pd.read_parquet(PATH_DF_CONSOLIDADO)
df_consolidado.head(5)

,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,amscPrsovDrivF-2b,basBleedLowPressF-1a,basBleedLowPressF-2b,basBleedLowTempF-1a,basBleedLowTempF-2b,...,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a,message0418DAA-1,message0422DAA-1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.34375,0.37500,0.12500,0.12500,3002.0,3002.0,2.000,0.0,0.0
1,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.03125,0.03125,0.03125,0.03125,3041.0,3041.0,2.125,0.0,0.0
2,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.03125,0.03125,0.03125,0.03125,5151.0,5151.0,2.250,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.03125,0.03125,0.03125,0.03125,20684.0,20684.0,2.250,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.34375,0.43750,0.28125,0.28125,3481.0,3481.0,2.250,0.0,0.0


In [9]:
# Seleçao de colunas para normalização
# Copia dessas colunas para outro dataframe

collums_normal = [
    'amscHprsovDrivF-1a',
    'amscHprsovDrivF-1b',
    'amscHprsovDrivF-2b',
    'amscPrsovDrivF-1a',
    'amscPrsovDrivF-1b', 
    'amscPrsovDrivF-2b',
    'basBleedLowPressF-1a',
    'basBleedLowPressF-2b',
    'basBleedLowTempF-1a',
    'basBleedLowTempF-2b',
    'basBleedOverPressF-1a', 
    'basBleedOverPressF-2b',
    'basBleedOverTempF-1a', 
    'basBleedOverTempF-2b',
    'bleedFavTmCmd-1a', 
    'bleedFavTmCmd-1b',
    'bleedFavTmCmd-2a', 
    'bleedFavTmCmd-2b', 
    'bleedFavTmFbk-1a',
    'bleedFavTmFbk-1b', 
    'bleedFavTmFbk-2b', 
    'bleedHprsovCmdStatus-1a',
    'bleedHprsovCmdStatus-1b', 
    'bleedHprsovCmdStatus-2a',
    'bleedHprsovCmdStatus-2b', 
    'bleedHprsovOpPosStatus-1a',
    'bleedHprsovOpPosStatus-1b', 
    'bleedHprsovOpPosStatus-2a',
    'bleedHprsovOpPosStatus-2b', 
    'bleedMonPress-1a',
    'bleedMonPress-1b', 
    'bleedMonPress-2a', 
    'bleedMonPress-2b',
    'bleedOnStatus-1a', 
    'bleedOnStatus-1b', 
    'bleedOnStatus-2b',
    'bleedOverpressCas-2a', 
    'bleedOverpressCas-2b',
    'bleedPrecoolDiffPress-1a', 
    'bleedPrecoolDiffPress-1b',
    'bleedPrecoolDiffPress-2a', 
    'bleedPrecoolDiffPress-2b',
    'bleedPrsovClPosStatus-1a',
    'bleedPrsovClPosStatus-2a',
    'bleedPrsovFbk-1a',
    # 'bleedPrsovFbk-1b',
    # 'bleedPrsovFbk-2b',
]

df_normal = df_consolidado[collums_normal].copy()

df_normal.head(5)


,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,amscPrsovDrivF-2b,basBleedLowPressF-1a,basBleedLowPressF-2b,basBleedLowTempF-1a,basBleedLowTempF-2b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.34375,0.37500,0.12500,0.12500,3002.0,3002.0,2.000
1,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.03125,0.03125,0.03125,0.03125,3041.0,3041.0,2.125
2,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.03125,0.03125,0.03125,0.03125,5151.0,5151.0,2.250
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.03125,0.03125,0.03125,0.03125,20684.0,20684.0,2.250
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.34375,0.43750,0.28125,0.28125,3481.0,3481.0,2.250


In [27]:
# normalização do dataframe normal
DF_NORMAL = df_normal[collums_normal]

DF_NORMAL = (DF_NORMAL - DF_NORMAL.min()) / (DF_NORMAL.max() - DF_NORMAL.min())
DF_NORMAL = DF_NORMAL.fillna(0)

display(DF_NORMAL)

,amscHprsovDrivF-1a,amscHprsovDrivF-1b,amscHprsovDrivF-2b,amscPrsovDrivF-1a,amscPrsovDrivF-1b,amscPrsovDrivF-2b,basBleedLowPressF-1a,basBleedLowPressF-2b,basBleedLowTempF-1a,basBleedLowTempF-2b,...,bleedOnStatus-2b,bleedOverpressCas-2a,bleedOverpressCas-2b,bleedPrecoolDiffPress-1a,bleedPrecoolDiffPress-1b,bleedPrecoolDiffPress-2a,bleedPrecoolDiffPress-2b,bleedPrsovClPosStatus-1a,bleedPrsovClPosStatus-2a,bleedPrsovFbk-1a
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.066225,0.072848,0.019868,0.019868,0.038178,0.025045,0.004047
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.038875,0.025374,0.004415
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.076589,0.043167,0.004783
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.354222,0.174154,0.004783
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.066225,0.086093,0.052980,0.052980,0.046740,0.029085,0.004783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.232191,0.0,0.0,0.582781,0.589404,0.629139,0.629139,0.036731,0.031041,0.801692
1269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.405217,0.0,0.0,0.609272,0.589404,0.602649,0.602649,0.037767,0.039567,0.000736
1270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.374120,0.0,0.0,0.589404,0.602649,0.602649,0.602649,0.040681,0.055066,0.000736
1271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.387396,0.0,0.0,0.596026,0.576159,0.609272,0.609272,0.035533,0.035704,0.799485


In [ ]:
# integração dos valores do df normal para o df consolidado

df_consolidado[collums_normal] = DF_NORMAL
display(df_consolidado)

<h4>CÓDIGOS PARA USO POSTERIOR</h4>

In [ ]:
unique_values = {}
for column in df.columns:
      unique_values[column] = df[column].unique()
unique_values

In [ ]:
# Descarting columns that are not needed
df = df.loc[:, columns_needed]